# Graph Neural Networks

歴史的に、機械学習で分子を扱う際の最大の課題は、記述子の選定とその計算でした。グラフニューラルネットワーク（GNN）は、グラフを入力とするディープニューラルネットワークの一種であり、GNNは分子を直接入力として受け取ることができるため、記述子に頭を悩ませる必要がありません。

<!-- Historically, the biggest difficulty for machine learning with molecules was the choice and computation of "descriptors". Graph neural networks (GNNs) are a category of deep neural networks whose inputs are graphs and provide a way around the choice of descriptors. A GNN can take a molecule directly as input. -->


```{admonition} 本章の想定読者と目的
この章では、{doc}`layers` と {doc}`../ml/regression` の内容は前提としています。本章ではグラフやGNNの定義から説明しますが、グラフやニューラルネットワークの基本的な概念について予め慣れているとなお良いでしょう。この章を学ぶことで、以下ができるようになります：

  * 分子をグラフで表現できる
  * 一般的なGNNのアーキテクチャを議論したり、GNNの種類を理解できる 
  * GNNを構築し、ラベルの種類に応じた読み出し関数（read-out function）を選択できる
  * グラフ、エッジ、ノードの特徴をそれぞれ区別できる
  * GNNをエッジ更新、ノード更新、集約の各ステップに分けて定式化できる
```

GNNはグラフを入力および出力するために特別に設計されたレイヤーです。GNNについてのレビューは複数執筆されており、例えば Dwivedi *et al.*{cite}`dwivedi2020benchmarking`, Bronstein *et al.*{cite}`bronstein2017geometric`, Wu *et al.*{cite}`wu2020comprehensive` などが挙げられます。
GNNは、粗視化分子動力学シミュレーション {cite}`li2020graph` からNMRの化学シフト予測 {cite}`yang2020predicting` 、固体のダイナミクスのモデリング {cite}`xie2019graph` まで、あらゆるアプリケーションに適用できます。
GNNについて深く踏み込む前に、まずグラフがコンピュータ上でどのように表現され、分子がどのようにグラフに変換されるか理解しましょう。

グラフとGNNについてのインタラクティブな入門資料が、 [distill.pub](https://distill.pub/2021/gnn-intro/) {cite}`sanchez-lengeling2021a` で提供されています。現在のGNNの研究のほとんどは、グラフに特化したディープラーニングライブラリを用いて行われており、2022年現在最も代表的なライブラリは [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/), [Deep Graph library](https://www.dgl.ai/), [Spektral](https://graphneural.network/), [TensorFlow GNNS](https://github.com/tensorflow/gnn) などです。

## グラフの表現

グラフ $\mathbf{G}$ は、ノード $\mathbf{V}$ およびエッジ $\mathbf{E}$　の集合です。 我々のセッティングでは、ノード $i$ はベクトル $\vec{v}_i$ で定義されるので、ノードの集合はランク2のテンソルとして表現できます。
エッジは隣接行列（adjacency matrix） $\mathbf{E}$ で表現され、もし $e_{ij} = 1$ であればノード $i$ と $j$ がエッジで結合しているとみなされます。 
グラフを扱う多くの分野において、簡単のため、グラフはしばしば有向非巡回グラフ（エッジには向きがあるが、一周して元のノードには戻らない）であると仮定されます。しかし、分子において結合には向きが無く、輪を持つ（巡回する）場合もあることに注意してください。化学結合において向きの概念はないことから、我々が扱う隣接行列は常に対称（$e_{ij} = e_{ji}$）となります。また、しばしばエッジ自身も特徴を持つ場合があり、 $e_{ij}$ 自体をベクトルとすることで表現します。この場合は隣接行列はランク3のテンソルとなります。エッジ特徴の例としては、共有結合の次数や、2つのノード間の距離（＝原子間距離）などが挙げられます。

```{figure} ./methanol.jpg
----
name: methanol
width: 400px
----
グラフに変換できるよう、メタノールの各原子に番号を割り当てた
```

```{margin} one-hot
one-hotベクトルとは、1つの要素が1で、それ以外の要素は全て0であるベクトルのことと思い出してください。例えば `[0, 1, 0, 0]`
1の要素がクラスを表します。今回の場合、クラスとは元素のことであり、各原子の元素をone-hotベクトルで表しています。
```

では、分子からどのようにグラフを構築できるか見てみましょう。例として、メタノールを考えます（ {numref}`methanol` ）。ノードとエッジを定義するため、便宜的に各原子に番号を振りました。まずはじめはノード特徴を考えます。ノードの特徴量には何を使っても良いのですが、多くの場合、one-hotエンコーディングされた特徴ベクトルを使うことになるでしょう：

| Node | C  | H  | O  |
|:-----|----|----|---:|
| 1    | 0  | 1  |  0 |
| 2    | 0  | 1  |  0 |
| 3    | 0  | 1  |  0 |
| 4    | 1  | 0  |  0 |
| 5    | 0  | 0  |  1 |
| 6    | 0  | 1  |  0 |

$\mathbf{V}$ が、これらのノードについての結合された特徴ベクトルになります。このグラフの近接行列 $\mathbf{E}$ は次のようになるでしょう：

|    | 1  | 2  | 3  | 4  | 5  | 6  | 
|:---|----|----|----|----|----|---:|
| 1  | 0  | 0  | 0  | 1  | 0  |  0 |
| 2  | 0  | 0  | 0  | 1  | 0  |  0 |
| 3  | 0  | 0  | 0  | 1  | 0  |  0 |
| 4  | 1  | 1  | 1  | 0  | 1  |  0 |
| 5  | 0  | 0  | 0  | 1  | 0  |  1 |
| 6  | 0  | 0  | 0  | 0  | 1  |  0 |


多少時間をかけてもよいので、これら2つをしっかりと理解してください。例えば、1,2,3行目については、4列目の成分だけが0でないことに注目してください。これは、原子1〜3は炭素（原子4）にのみ結合しているからです。また、原子は自分自身とは結合できないので、対角成分は常に0になります。

分子だけでなく、結晶構造についても似た方法でグラフ化できます。これについては Xie et al.による {cite}`Xie2018Crystal` を参照してください。

それでは、SMILESによる分子の文字列表現をグラフに変換する関数を定義するところから始めましょう。

## このノートブックの動かし方

このページ上部の &nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp; を押すと、このノートブックがGoogle Colab.で開かれます。必要なパッケージのインストール方法については以下を参照してください。

````{tip} My title
:class: dropdown
必要なパッケージをインストールするには、新規セルを作成して次のコードを実行してください。

```
!pip install dmol-book
```
もしインストールがうまくいかない場合、パッケージのバージョン不一致が原因である可能性があります。動作確認がとれた最新バージョンの一覧は[ここ](https://github.com/whitead/dmol-book/blob/master/package/requirements.txt)から参照できます

````

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import tensorflow as tf
import pandas as pd
import rdkit, rdkit.Chem, rdkit.Chem.rdDepictor, rdkit.Chem.Draw
import networkx as nx
import dmol

In [ ]:
soldata = pd.read_csv(
    "https://github.com/whitead/dmol-book/raw/master/data/curated-solubility-dataset.csv"
)
np.random.seed(0)
my_elements = {6: "C", 8: "O", 1: "H"}

下の非表示セルでは、関数 `smiles2graph` を定義しています。この関数は元素C, H, Oについてone-hotなノード特徴ベクトルを生成します。また同時に、このone-hotベクトルを特徴ベクトルとする隣接テンソルを生成します。

In [ ]:
def smiles2graph(sml):
    """Argument for the RD2NX function should be a valid SMILES sequence
    returns: the graph
    """
    m = rdkit.Chem.MolFromSmiles(sml)
    m = rdkit.Chem.AddHs(m)
    order_string = {
        rdkit.Chem.rdchem.BondType.SINGLE: 1,
        rdkit.Chem.rdchem.BondType.DOUBLE: 2,
        rdkit.Chem.rdchem.BondType.TRIPLE: 3,
        rdkit.Chem.rdchem.BondType.AROMATIC: 4,
    }
    N = len(list(m.GetAtoms()))
    nodes = np.zeros((N, len(my_elements)))
    lookup = list(my_elements.keys())
    for i in m.GetAtoms():
        nodes[i.GetIdx(), lookup.index(i.GetAtomicNum())] = 1

    adj = np.zeros((N, N, 5))
    for j in m.GetBonds():
        u = min(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        v = max(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        order = j.GetBondType()
        if order in order_string:
            order = order_string[order]
        else:
            raise Warning("Ignoring bond order" + order)
        adj[u, v, order] = 1
        adj[v, u, order] = 1
    return nodes, adj

In [ ]:
nodes, adj = smiles2graph("CO")
nodes

## グラフニューラルネットワーク

グラフニューラルネットワーク (GNN) は、次の2つの特徴を持つニューラルネットワークです。

1. 入力がグラフである
2. 出力は順序等価（permutation equivariant）である

最初の点は明らかですが、2つめの特徴は説明が必要でしょう。まず、グラフの並べ替えとは、ノードを並べ替えることを意味します。
上のメタノールの例では、炭素を原子4ではなく原子1にすることも簡単にできます。その場合、新しい隣接行列は次のようになります：

|    | 1  | 2  | 3  | 4  | 5  | 6  | 
|:---|----|----|----|----|----|---:|
| 1  | 0  | 1  | 1  | 1  | 1  |  0 |
| 2  | 1  | 0  | 0  | 0  | 0  |  0 |
| 3  | 1  | 0  | 0  | 0  | 0  |  0 |
| 4  | 1  | 0  | 0  | 0  | 1  |  0 |
| 5  | 1  | 0  | 0  | 0  | 0  |  1 |
| 6  | 0  | 0  | 0  | 0  | 1  |  0 |

この隣接行列の交換に正しく対応してGNNの出力が変換するなら、そのGNNは順序等価であると言えます。部分電荷や化学シフトのように原子ごとに定義される量をモデリングする場合、このような順序不変の仕組みは不可欠です。つまり、もし原子を入力する順序を変えれば、予測される部分電荷の順序も同様に変わってほしいのです。

もちろん、溶解度やエネルギーのように分子全体の特性をモデリングしたい場合もあります。これらの量は原子の順番を変えても **不変（invariant）** であるべきです。順序について等価（equivariant）なモデルを不変（invariant）にするため、後に定義するリードアウト（read-out）を使います。等価性についてのより詳細な議論は {doc}`data` を参照してください。


### シンプルなGNN

我々はこれからGNNに言及しますが、実際にはGNN全体ではなく特定のレイヤーのことを指します。多くのGNNはグラフを取り扱うために特別に設計されたレイヤーを備えており、通常はこのレイヤーについてのみ関心を持ちます。それでは、GNNの簡単なレイヤーの例を見てみましょう:

\begin{equation}
f_k = \sigma\left( \sum_i \sum_j v_{ij}w_{jk}  \right)
\end{equation}

この式は、まず各ノード（$v_{jij}$）の特徴に学習可能な重み $w_{jk}$ をかけた後、全てのノードの特徴を合計し、活性化を適用することを表しています。この操作により、グラフに対して1つの特徴ベクトルが得られます。では、この式は順序等価でしょうか？答えはYesです。なぜならこの式においてノードインデックスはインデックス $i$ であり、出力に影響を与えることなく順序の並べ替えが可能であるためです。

では次に、この例と似ているが順序等価ではない例を見てみましょう。

\begin{equation}
f_k = \sigma\left( \sum_i v_{ij}w_{ik}  \right)
\end{equation}

これは小さな変化です。いま、ノードごとに1つの重みベクトルがあります。したがって、学習可能な重みはノードの順序に依存します。次に、ノードの順序を入れ替えると、学習した重みはノードに対応しなくなります。よって、ノード（＝原子）の順番を変えた2つのメタノール分子を入力すると、異なる出力が得られます。実際のところ、この単純な例は2つの点で実際のGNNと異なります。1つめは単一の特徴ベクトルを出力してノードごとの情報を捨ててしまっている点、2つめは隣接行列を使用しない点です。では、順序等価を維持し、かつこれら2つの性質を備えた実際のGNNを見てみましょう。

## Kipf & Welling GCN

初期に人気があったGNNの一つは、Kipf & Welling graph convolutional network (GCN) {cite}`kipf2016semi` です。GCNをGNNの広いクラスの一つとして考える人もいますが、本書でGCNとは特にKipf & Welling GCNを指すものとします。
Thomas Kipfは、[GCNの優れた紹介記事](https://tkipf.github.io/graph-convolutional-networks/)を書いています。

GCNレイヤーへの入力はノードおよびエッジの集合（訳注：各ノードおよびエッジはベクトルで表現されるので、これらの集合はテンソルです） $\mathbf{V}$, $\mathbf{E}$ で、出力は更新されたノードの集合 $\mathbf{V}'$ です。 各ノード特徴の更新は、 $\mathbf{E}$ により表現される近傍ノードの特徴ベクトルを平均することでなされます。
<!-- todo: Eについての記述を誤読していないか見直す。原文は The way it updates a node feature vector is by averaging the feature vectors of its neighbors, as determined by $\mathbf{E}$. -->
近傍ノードの情報を平均することで、GCNレイヤーはノードについて順序等価になっています。近傍について平均するという操作そのものは学習できないので、特徴ベクトルを加算してからノードの次数（結合している近傍ノード数）で除算することで計算します。平均をとる前に、学習可能な重み行列を近傍特徴に掛けることにします。これによりGCNがデータから学習することが可能になります（訳注：言い換えれば、GCNの学習とはこの重み行列の要素を最適化することです）。この操作は次のように記述されます：

$$
v_{il} = \sigma\left(\frac{1}{d_i}e_{ij}v_{jk}w_{lk}\right)
$$ (gcn)

$i$ は着目しているノード、 $j$ はその近傍インデックス, $k$ はノードの入力特徴、 $l$ はノードの出力特徴、 $d_i$ はノードの次数（次数で割ることで単なる加算ではなく平均になります）、 $e_{ij}$ は、全ての非近傍ノードが $v_{jk}$ ゼロになるよう近傍と非近傍ノードを分離する項、 $\sigma$ は活性化関数、 $w_{lk}$ は学習可能な重みです。 
この式はとても長いように見えますが、実際にやっていることは、近傍同士の平均に学習可能な重み行列を追加しただけです。この式のよくある拡張として、各ノードの近傍として自分自身も加える場合があります。これはノードの出力特徴 $v_{il}$ が入力特徴 $v_{ik}$ に依存するようにするためです。しかし、我々はこのために上の式を修正する必要はありません。もっとシンプルなやり方として、データの前処理で恒等行列を加算し、近接行列の対角成分を $0$ ではなく$1$ にしてやればよいのです。

GCNについての理解を深めることは、他の種類のGNNを理解するために重要です。ここでは2つのポイントを押さえてください。まずGCNレイヤーは、ノードとその近傍の間で”通信”する方法と見ることができます。ノード $i$ についての出力は、そのすぐ隣のノードにのみ依存することになりますが、化学の場合、これでは不十分です。より遠方のノードの情報を取り込むために、我々は複数のGCNレイヤーを重ねることができます。もし2つのGCNレイヤーがあれば、ノード $i$ の出力は、その隣の隣のノードの情報も含むことになります。
GCNで理解すべきもう一つの点は、ノード特徴を平均化するステップが2つの目的を達していることです：i)近傍ノードの順序を無視することで順序に対して等価となる、ii)ノード特徴のデータのスケール変化を防ぐ。単にノード特徴の和を取った場合、(i)は実現しますが、各レイヤーを通すごとにノード特徴のデータの値が大きくなってしまいます。もちろん、特徴のスケールを揃えるために各GCNレイヤーの後でバッチ平均化（batch normalization）レイヤーを通すという対処もありますが、平均化はよりシンプルです。

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT
from myst_nb import glue
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage


def draw_vector(x, y, s, v, ax, cmap, **kwargs):
    x += s / 2
    y += s / 2
    for vi in v:
        if cmap is not None:
            ax.add_patch(
                mpl.patches.Rectangle((x, y), s * 1.5, s, facecolor=cmap(vi), **kwargs)
            )
        else:
            ax.add_patch(
                mpl.patches.Rectangle(
                    (x, y), s * 1.5, s, facecolor="#FFF", edgecolor="#333", **kwargs
                )
            )
        ax.text(
            x + s * 1.5 / 2,
            y + s / 2,
            "{:.2f}".format(vi),
            verticalalignment="center",
            horizontalalignment="center",
        )
        y += s


def draw_key(x, y, s, v, ax, cmap, **kwargs):
    x += s / 2
    y += s / 2
    for vi in v:
        ax.add_patch(
            mpl.patches.Rectangle((x, y), s * 1.5, s, facecolor=cmap(1.0), **kwargs)
        )
        ax.text(
            x + s * 1.5 / 2,
            y + s / 2,
            vi,
            verticalalignment="center",
            horizontalalignment="center",
        )
        y += s
    ax.text(
        x, y + s / 2, "Key:", verticalalignment="center", horizontalalignment="left"
    )


def draw(
    nodes, adj, ax, highlight=None, key=False, labels=None, mask=None, draw_nodes=None
):
    G = nx.Graph()
    for i in range(adj.shape[0]):
        for j in range(adj.shape[0]):
            if np.any(adj[i, j]):
                G.add_edge(i, j)
    if mask is None:
        mask = [True] * len(G)
    if draw_nodes is None:
        draw_nodes = nodes
    # go from atomic number to element
    elements = np.argmax(draw_nodes, axis=-1)
    el_labels = {i: list(my_elements.values())[e] for i, e in enumerate(elements)}
    try:
        pos = nx.nx_agraph.graphviz_layout(G, prog="sfdp")
    except ImportError:
        pos = nx.spring_layout(G, iterations=100, seed=4, k=1)
    pos = nx.rescale_layout_dict(pos)
    c = ["white"] * len(G)
    all_h = []
    if highlight is not None:
        for i, h in enumerate(highlight):
            for hj in h:
                c[hj] = "C{}".format(i + 1)
                all_h.append(hj)
    nx.draw(G, ax=ax, pos=pos, labels=el_labels, node_size=700, node_color=c)
    cmap = plt.get_cmap("Wistia")
    for i in range(len(G)):
        if not mask[i]:
            continue
        if i in all_h:
            draw_vector(*pos[i], 0.15, nodes[i], ax, cmap)
        else:
            draw_vector(*pos[i], 0.15, nodes[i], ax, None)
    if key:
        draw_key(-1, -1, 0.15, my_elements.values(), ax, cmap)
    if labels is not None:
        legend_elements = []
        for i, l in enumerate(labels):
            p = mpl.lines.Line2D(
                [0], [0], marker="o", color="C{}".format(i + 1), label=l, markersize=15
            )
            legend_elements.append(p)
        ax.legend(handles=legend_elements)
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_facecolor("#f5f4e9")


fig = plt.figure(figsize=(8, 5))
draw(nodes, adj, plt.gca(), highlight=[[1], [5, 0]], labels=["center", "neighbors"])
fig.set_facecolor("#f5f4e9")
glue("dframe", plt.gcf(), display=False)

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT
fig, axs = plt.subplots(1, 2, squeeze=True, figsize=(14, 6), dpi=100)
order = [5, 1, 0, 2, 3, 4]
time_per_node = 2
last_layer = [0]
layers = 2
input_nodes = np.copy(nodes)
fig.set_facecolor("#f5f4e9")


def make_frame(t):
    axs[0].clear()
    axs[1].clear()

    layer_i = int(t / (time_per_node * len(order)))
    axs[0].set_title(f"Layer {layer_i + 1} Input")
    axs[1].set_title(f"Layer {layer_i + 1} Output")

    flat_adj = np.sum(adj, axis=-1)
    out_nodes = np.einsum(
        "i,ij,jk->ik",
        1 / (np.sum(flat_adj, axis=1) + 1),
        flat_adj + np.eye(*flat_adj.shape),
        nodes,
    )

    if last_layer[0] != layer_i:
        print("recomputing")
        nodes[:] = out_nodes
        last_layer[0] = layer_i

    t -= layer_i * time_per_node * len(order)
    i = order[int(t / time_per_node)]
    print(last_layer, layer_i, i, t)
    mask = [False] * nodes.shape[0]
    for j in order[: int(t / time_per_node) + 1]:
        mask[j] = True
    print(mask, i)
    neighs = list(np.where(adj[i])[0])
    if (t - int(t / time_per_node) * time_per_node) >= time_per_node / 4:
        draw(
            nodes,
            adj,
            axs[0],
            highlight=[[i], neighs],
            labels=["center", "neighbors"],
            draw_nodes=input_nodes,
        )
    else:
        draw(
            nodes,
            adj,
            axs[0],
            highlight=[[i]],
            labels=["center", "neighbors"],
            draw_nodes=input_nodes,
        )
    if (t - int(t / time_per_node) * time_per_node) < time_per_node / 2:
        mask[j] = False
    draw(
        out_nodes,
        adj,
        axs[1],
        highlight=[[i]],
        key=True,
        mask=mask,
        draw_nodes=input_nodes,
    )
    fig.set_facecolor("#f5f4e9")
    return mplfig_to_npimage(fig)


animation = VideoClip(make_frame, duration=time_per_node * nodes.shape[0] * layers)

animation.write_gif("../_static/images/gcn.gif", fps=2)

```{glue:figure} dframe
----
name: dframe
----
グラフ畳み込みレイヤーの中間ステップ。3次元ベクトルはノード特徴であり、初期値は水素を表すone-hotベクトル `[1.00, 0.00, 0.00]` です。この中心ノードは、隣接するノードの特徴を平均することで更新されていきます。
```


GCNレイヤーを理解しやすくするため、 {numref}`dframe` を見てください。これはGCNレイヤーの中間ステップを表しています。各ノードの特徴は、ここではone-hot encodingされたベクトルとして表現されています。{numref}`gcnanim` のアニメーションは、近傍特徴についての平均化プロセスを表しています。このアニメーションでは、わかりやすくするために学習可能な重みと活性化関数は記述されていません。このアニメーションは2層目のGCNレイヤーでも繰り返されることに注意してください。分子中に酸素原子が含まれるという”情報”が、2層目ではじめて各原子に伝搬される様子をよく見てください。全てのGNNは似たようなアプローチで動作するので、このアニメーションの内容はとても大切です。ぜひ、よく理解してください。


```{figure} ../_static/images/gcn.gif
----
name: gcnanim
----
グラフ畳み込み層の動作のアニメーション。左が入力、右が出力ノードの特徴です。2つの層が表示されていることに注意してください（タイトルが変わることに注意して見てください）。アニメーションが進むにつれて、近傍ノードの平均化によって、原子についての情報がどのように分子内を伝播していくかがわかることでしょう。つまり、酸素は単なる酸素から、CとHに結合した酸素、HとCH3に結合した酸素へと変化していくのです。図中の色は数値に対応しています。
```


### GCNの実装

それでは、GCNのテンソル実装を作りましょう。ここでは一旦、活性化関数および学習可能な重みについては省略します。
まず最初に、ランク2の隣接行列を計算する必要があります。上の `smiles2graph` コードは、特徴ベクトルを用いて隣接行列を計算します。この計算は簡単です。同時に恒等行列を加えることにします（訳注：この恒等行列を加算する操作は、上で述べたように自分自身を隣接ノードとして取り扱うための工夫です）。


In [ ]:
nodes, adj = smiles2graph("CO")
adj_mat = np.sum(adj, axis=-1) + np.eye(adj.shape[0])
adj_mat

各ノードの次数を計算するために、また別な縮約操作を行います：

In [ ]:
degree = np.sum(adj_mat, axis=-1)
degree

これでノードの更新操作の準備ができました。アインシュタインの縮約記法を使って更新操作を表現すると次のようになります

In [ ]:
print(nodes[0])
# note to divide by degree, make the input 1 / degree
new_nodes = np.einsum("i,ij,jk->ik", 1 / degree, adj_mat, nodes)
print(new_nodes[0])

これをKerasのLayerとして実装するには、上記のコードを新しいLayerのサブクラスとして記述する必要があります。今回のコードは比較的簡単ですが、Kerasの関数名とLayerクラスについて、[このチュートリアル](https://keras.io/guides/making_new_layers_and_models_via_subclassing/)を読んで学ぶことを推奨します。主な変更点は、学習可能なパラメータ `self.w` を作って {obj}`tf.einsum` の中で用いること、活性化関数 `self.activation` を用いること、そして新しいノード特徴と隣接行列を出力することの3点です。隣接行列を出力する理由は、隣接行列を毎回渡すことなく、複数のGCNレイヤーをスタックできるようにするためです。

In [ ]:
class GCNLayer(tf.keras.layers.Layer):
    """Implementation of GCN as layer"""

    def __init__(self, activation=None, **kwargs):
        # constructor, which just calls super constructor
        # and turns requested activation into a callable function
        super(GCNLayer, self).__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        # create trainable weights
        node_shape, adj_shape = input_shape
        self.w = self.add_weight(shape=(node_shape[2], node_shape[2]), name="w")

    def call(self, inputs):
        # split input into nodes, adj
        nodes, adj = inputs
        # compute degree
        degree = tf.reduce_sum(adj, axis=-1)
        # GCN equation
        new_nodes = tf.einsum("bi,bij,bjk,kl->bil", 1 / degree, adj, nodes, self.w)
        out = self.activation(new_nodes)
        return out, adj

上記のコードの大半はKeras/TFに固有のもので、変数を適切な場所に配置しています。ここで重要なのは2行だけです。1つめは、隣接行列の列について合計することでグラフの次数を計算する操作です：

```python
degree = tf.reduce_sum(adj, axis=-1)
```

2つめの重要な行は、GCN方程式 {eq}`gcn` を計算する部分です（ここでは活性化は省略しています）：

```python
new_nodes = tf.einsum("bi,bij,bjk,kl->bil", 1 / degree, adj, nodes, self.w)
```

これで、いま実装したGCNレイヤーを試すことができるようになりました：

In [ ]:
gcnlayer = GCNLayer("relu")
# we insert a batch axis here
gcnlayer((nodes[np.newaxis, ...], adj_mat[np.newaxis, ...]))

これにより (1) 新しいノード特徴、(2) 隣接行列が出力されます。このレイヤーを積み重ねて、GCNを複数回適用できることを確認しましょう。

In [ ]:
x = (nodes[np.newaxis, ...], adj_mat[np.newaxis, ...])
for i in range(2):
    x = gcnlayer(x)
print(x)

うまくいきました！しかし、なぜゼロの値があるのでしょうか？これはおそらく、出力に負の値が含まれており、それがReLU活性化を通した際に0になったためでしょう。これはモデルの訓練が不十分なために起きていると考えられ、訓練を重ねることで解決するでしょう

## 例：溶解度予測
次に、GCNによる溶解度の予測について説明します。以前に、分子データセットに含まれている特徴量を使って予測モデルを組んだことを思い出してください。いま我々はGCNを使えるようになったので、特徴量に頭を悩ますことなく、分子構造を直接ニューラルネットワークに入力できるようになりました。GCNレイヤーは各ノードについての特徴を出力しますが、溶解度を予測するためには、グラフ全体についての特徴を得る必要があります。このプロセスをさらに洗練する方法については後で説明しますが、ここでは、GCNレイヤー後のすべてのノード特徴の平均を使うことにします。これにより単純かつ順序不変に、ノード特徴をグラフ特徴に変換することができます。この実装は次のとおりです：

In [ ]:
class GRLayer(tf.keras.layers.Layer):
    """A GNN layer that computes average over all node features"""

    def __init__(self, name="GRLayer", **kwargs):
        super(GRLayer, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        nodes, adj = inputs
        reduction = tf.reduce_mean(nodes, axis=1)
        return reduction

上のコードで重要な点は、ノードについて平均をとっている(`axis=1`)次の部分だけです：

```python
reduction = tf.reduce_mean(nodes, axis=1)
```

この溶解度予測器を完成させるため、いくつかの全結合層を追加して、回帰が行えることを確認しましょう。回帰の場合は最終層の出力がそのまま予測結果となるため、最終層には活性化を適用しないことに注意してください。このモデルは [Keras functional API](https://keras.io/guides/functional_api/) を使って実装されています。

In [ ]:
ninput = tf.keras.Input(
    (
        None,
        100,
    )
)
ainput = tf.keras.Input(
    (
        None,
        None,
    )
)
# GCN block
x = GCNLayer("relu")([ninput, ainput])
x = GCNLayer("relu")(x)
x = GCNLayer("relu")(x)
x = GCNLayer("relu")(x)
# reduce to graph features
x = GRLayer()(x)
# standard layers (the readout)
x = tf.keras.layers.Dense(16, "tanh")(x)
x = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=(ninput, ainput), outputs=x)

この100はどこから来たのでしょうか？その答えはデータセットに含まれる元素の数にあります。このデータセットは多数の元素を含むため、以前使ったサイズ3のone-hot encodingでは、全ての元素を表現できません。前回はC, H, Oの元素さえ表現できれば十分でしたが、今回はより多数の元素を扱う必要があります。そのため、one-hot encodingのサイズも100に増やすことにしました。これで最大100種類の元素を表現できます。この拡張のために、モデルだけでなく`smiles2graph`関数も更新することにしましょう。

In [ ]:
def gen_smiles2graph(sml):
    """Argument for the RD2NX function should be a valid SMILES sequence
    returns: the graph
    """
    m = rdkit.Chem.MolFromSmiles(sml)
    m = rdkit.Chem.AddHs(m)
    order_string = {
        rdkit.Chem.rdchem.BondType.SINGLE: 1,
        rdkit.Chem.rdchem.BondType.DOUBLE: 2,
        rdkit.Chem.rdchem.BondType.TRIPLE: 3,
        rdkit.Chem.rdchem.BondType.AROMATIC: 4,
    }
    N = len(list(m.GetAtoms()))
    nodes = np.zeros((N, 100))
    for i in m.GetAtoms():
        nodes[i.GetIdx(), i.GetAtomicNum()] = 1

    adj = np.zeros((N, N))
    for j in m.GetBonds():
        u = min(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        v = max(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        order = j.GetBondType()
        if order in order_string:
            order = order_string[order]
        else:
            raise Warning("Ignoring bond order" + order)
        adj[u, v] = 1
        adj[v, u] = 1
    adj += np.eye(N)
    return nodes, adj

In [ ]:
nodes, adj = gen_smiles2graph("CO")
model((nodes[np.newaxis], adj_mat[np.newaxis]))

```{margin}
GCNではエッジ特徴が使えないので、隣接テンソルの代わりに行列を使うよう変更を加えました。
```
このモデルは1つの数値（スカラー）を出力します。
さて、学習可能なデータセットを得るために、いくつか作業が必要です。このデータセットは少し複雑で、特徴はテンソル($mathbf{V}, \mathbf{E}$)のタプルなので、データセットは次のようなタプルのタプルになります： $\left((\mathbf{V}, \mathbf{E}), y\right)$ **generator**はPythonの関数で、値を繰り返し返すことができます。ここでは、学習データを1つずつ取り出すためにgeneratorを使います。続いて、これを `from_generator` {obj}`tf.data.Dataset` コンストラクタに渡します。このコンストラクタでは、入力データのshapeを明示的に指定する必要があります。

In [ ]:
def example():
    for i in range(len(soldata)):
        graph = gen_smiles2graph(soldata.SMILES[i])
        sol = soldata.Solubility[i]
        yield graph, sol


data = tf.data.Dataset.from_generator(
    example,
    output_types=((tf.float32, tf.float32), tf.float32),
    output_shapes=(
        (tf.TensorShape([None, 100]), tf.TensorShape([None, None])),
        tf.TensorShape([]),
    ),
)

ここまで来たらもう少しです。これで、いつものようにデータセットをtrain/val/testに分割できます。

In [ ]:
test_data = data.take(200)
val_data = data.skip(200).take(200)
train_data = data.skip(400)

そして、いよいよモデルの訓練です

In [ ]:
model.compile("adam", loss="mean_squared_error")
result = model.fit(train_data.batch(1), validation_data=val_data.batch(1), epochs=10)

In [ ]:
plt.plot(result.history["loss"], label="training")
plt.plot(result.history["val_loss"], label="validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

このモデルは明らかにアンダーフィットです。考えられる理由の一つはバッチサイズが1であることです。このモデルでは、原子個数を可変にした副作用としてバッチサイズを1より大きくできない制約があります。もう少し詳しく説明すると、任意のバッチサイズを入力できるようにすると、原子個数とバッチサイズの2つが不定となります。Keras/tensorflowは入力データのshapeに未知の次元が2つ以上ある場合データを処理できないため、バッチサイズを1に固定することで対処しています。ここでは扱いませんが、この問題を回避してバッチサイズを1より大きくするための標準的トリックは、複数の分子を（分子間の結合がない）1つのグラフにまとめてしまうことです。これによりデータの次元はそのままに、分子をバッチ処理することができます。

それではパリティプロットで予測精度を確認してみましょう。

In [ ]:
yhat = model.predict(test_data.batch(1), verbose=0)[:, 0]
test_y = [y for x, y in test_data]
plt.figure()
plt.plot(test_y, test_y, "-")
plt.plot(test_y, yhat, ".")
plt.text(
    min(test_y) + 1,
    max(test_y) - 2,
    f"correlation = {np.corrcoef(test_y, yhat)[0,1]:.3f}",
)
plt.text(
    min(test_y) + 1,
    max(test_y) - 3,
    f"loss = {np.sqrt(np.mean((test_y - yhat)**2)):.3f}",
)
plt.title("Testing Data")
plt.show()

## Message PassingとGCN

より広い意味でGCNレイヤーを捉えると、GCNレイヤーは"message-passing"レイヤーの一つと言えます。GCNでは、まず近傍ノードからやってくるメッセージを処理します：

\begin{equation}
\vec{e}_{{s_i}j} = \vec{v}_{{s_i}j} \mathbf{W}
\end{equation}

ここで $v_{{s_i}j}$ は ノード $i$ の $j$ 番目の近傍です。 $s_i$ は $i$ に対するセンダー（送信者）です。 これはGCNがどのようにメッセージを計算するか示したものですが、やっていることは単純で、各近傍ノードの特徴に重み行列をかけているだけです。ノード $i$ に向かうメッセージ $\vec{e}_{{s_i}j}$ を得た後、これらのメッセージをノードの順番に対して不変な関数を用いて集約します：

\begin{equation}
\vec{e}_{i} = \frac{1}{|\vec{e}_{{s_i}j}|}\sum \vec{e}_{{s_i}j} 
\end{equation}

上で扱ったように、GCNではこの集約は単なる平均ですが、任意の（例えば学習可能な）順序不変の関数を使うこともできます：

```{margin}
訳注：「学習可能な関数」という表現が時々出てきますが、これはtrainable functionの訳で「その関数自体を（具体的な関数形を明示的に記述するのではなく）MLP等の機械学習モデルとデータによりモデリングすることで、所望の性質を備えた関数を構成する」というアプローチにより構成された関数を指しています。
```

\begin{equation}
\vec{v}^{'}_{i} = \sigma(\vec{e}_i)
\end{equation}

$v^{'}$ は新しいノード特徴を示しています。これは単純に、集約された後で活性化関数を適用したメッセージです。このように書き出すことで、これらの手順にいくらでも小さな変更が加えられることに気づいたのではないでしょうか。Gilmerらによる重要な論文 {cite}`gilmer2017neural` では、いくつかの選択肢を検討し、このメッセージパッシングレイヤーの基本的なアイディアが、量子力学に基づいて分子エネルギーを予測するタスクでうまくいくことが述べられています。GCN式に変更を加えた例としては、近傍メッセージの計算においてエッジ特徴を含めたり、単に $\sigma$ で和をとる代わりに全結合層を使うといった試みが挙げられます。
これらから、GCNは、メッセージパッシンググラフニューラルネットワーク（MPNNと略されることもあります）の一種と考えることができます。


## Gated Graph Neural Network

メッセージパッシングレイヤーの最も有名な亜種の一つは、 **gated graph neural network** (GGN) {cite}`li2015gated` です。これは最後の式のノード更新を次で置き換えたものです：

\begin{equation}
\vec{v}^{'}_{i} = \textrm{GRU}(\vec{v}_i, \vec{e}_i)
\end{equation}

$\textrm{GRU}(\cdot, \cdot)$ はゲート再帰ユニット（gated recurrent unit）{cite}`chung2014empirical` です。 GRU はバイナリ（入力引数を2つ持つ）ニューラルネットワークで、典型的には系列データのモデリングに使用されます。GCNと比較して、GGNの興味深い特徴は、（GRUからの）ノード更新において学習可能なパラメータを持つことで、より柔軟性を備えたモデルとなっていることです。GGNでは、GRUのパラメータは各層で共有されます（GRUを使って系列データをモデリングする方法と同じです）。パラメータ共有によるメリットは、学習すべきパラメータを増やすことなくGGNの層を無限に積み重ねられることです（各層で $\mathbf{W}$ を揃えることが前提です）。このため、GGNは大きなタンパク質や大きなユニットセルをもつ結晶構造のような、大きなグラフに適しています（訳注：GGNレイヤーを何層もスタックすることで、より遠くのノードからの情報を取り込むことができるためです）。

```{margin}
GNNにはよく"gated"という接頭語がついていますが、これはノードがGRUによって更新されることを意味します。
```


## Pooling
メッセージパッシングの観点、および一般にGNNSでは、近傍からのメッセージを結合する方法が重要なステップとなります。このステップは畳み込みニューラルネットワークで使われるプーリング層に似ているため、**プーリング**と呼ばれることもあります。畳み込みニューラルネットワークのプーリングと同じように、このために使用できる縮約操作は複数あります。一般的に、GNNのプーリングにはメッセージの合計または平均が使われますが、グラフ同型ネットワーク（Graph Isomorphism Networks） {cite}`xu2018powerful` のように非常に洗練された操作を用いることもできます。この他にも、例えば注意（attention）の章では自己注意（self-attention）を使う例を扱いますが、これらの操作もプーリングに使えます。プーリングのステップを色々と工夫したくなることもありますが、プーリング操作の選択はモデルの性能にとってそれほど重要ではないことが経験的に分かっています {cite}`luzhnica2019graph, mesquita2020rethinking`。プーリングの重要な特性は*順序不変性*で、集約操作はノード（プーリングの場合はエッジ）の順序に依存しない性質を備えることが望まれます。Grattarolaら {cite}`grattarola2021understanding`が、プーリング手法に関する最近のレビューを出版しています。

Daigavaneらの論文 {cite}`daigavane2021understanding` では、様々なプーリング戦略の比較と概要が視覚的に紹介されています。


## Readout Function

GNNの出力はグラフです（そう設計されているので当たり前ですが）。しかし、予測したいラベルもグラフであることは稀で、一般的にはラベルは各ノードまたはグラフ全体に対して付与されています。ノードラベルの例は原子の部分電荷、グラフラベルの例は分子のエネルギーです。GNNから出力されるグラフを、予測ターゲットであるノードラベルやグラフラベルに変換するプロセスを **読み出し （readout）** と呼びます。ノードラベルを予測する場合であれば、単純にエッジ特徴を捨てて、GNNから出力されるノード特徴ベクトルを予測結果として扱うことができます。この場合、出力層の前にいくつかの全結合層を挟むことが多いでしょう。

分子のエネルギーや実効電荷のようなグラフレベルのラベルを予測する場合、ノード/エッジ特徴をグラフラベルに変換するプロセスに注意が必要です。所望のshapeのグラフラベルを得るために、単純にノード特徴を全結合層に入力した場合、順序等価性が失われてしまいます（出力はノードラベルではなくグラフラベルなので、厳密には、順序等価性ではなく順序不変性です）。溶解度の例で用いた読み出しは、ノード特徴量に対して縮約操作を行うことでした。その後でグラフ特徴を全結合層に入力して予測結果を得ました。実は、これがグラフ特徴の読み出しを行う唯一の方法であることが示されています {cite}`zaheer2017deep` 。すなわち、グラフ特徴を得るためにノード特徴の縮約操作を行い、このグラフ特徴を全結合層に入力することで予測結果であるグラフラベルを得ます。各ノードの特徴量に対してそれぞれ全結合層を通す操作もできますが、ノード特徴への全結合層の適用はGNN内部で既に行われているので、あまりお勧めしません。このグラフ特徴の読み出しはDeepSetsと呼ばれることもあります。これは、特徴が集合（訳注：順序を持たず、個数が不定）として与えられる場合のために設計された、順序不変なアーキテクチャであるDeepSets {cite}`zaheer2017deep` と同じ形であるためです。

プーリングも読み出しも順序不変の関数が使われていることにお気づきでしょうか。したがって、DeepSetsはプーリングに、attentionは読み出しに使用することもできます。

### Intensive vs Extensive

回帰タスクでの読み出しにおいて考慮すべき重要な点の1つは、ラベルが **intensive** か **extensive** かです。Intensiveラベルは、ノード（＝原子）の数に依存しない値を持つラベルです。例えば、屈折率や溶解度などはIntensiveラベルです。Intensiveラベルの読み出しは、一般にノードの数に依存しないことが要請される。したがって、この場合の読み出しにおける縮約操作として、平均や最大をとる操作は適用可能ですが、ノード数により値が変わるため合計は適しません。対照的に、Extensiveラベルでは、（一般的には）読み出しの縮約操作には（ノード数を反映できるため）合計が適します。extensiveな分子特性の例には、生成エンタルピーが挙げられます。

## Battaglia General Equations

As you can see, message passing layers is a general way to view GNN layers. Battaglia *et al.* {cite}`battaglia2018relational` went further and created a general set of equations which captures nearly all GNNs. They broke the GNN layer equations down into 3 update equations, like the node update equation we saw in the message passing layer equations, and 3 aggregation equations (6 total equations). There is a new concept in these equations: graph feature vectors. Instead of having two parts to your network (GNN then readout), a graph level feature is updated at every GNN layer. The graph feature vector is a set of features which represent the whole graph or molecule. For example, when computing solubility it may have been useful to build up a per-molecule feature vector that is eventually used to compute solubility instead of having the readout. Any kind of per-molecule quantity, like energy, should be predicted with the graph-level feature vector. 

The first step in these equations is updating the edge feature vectors, written as $\vec{e}_k$, which we haven't seen yet:

$$
\vec{e}^{'}_k = \phi^e\left( \vec{e}_k, \vec{v}_{rk}, \vec{v}_{sk}, \vec{u}\right)
$$ (edge-update)

where $\vec{e}_k$ is the feature vector of edge $k$, $\vec{v}_{rk}$ is the receiving node feature vector for edge $k$, $\vec{v}_{sk}$ is the sending node feature vector for edge $k$, $\vec{u}$ is the graph feature vector, and $\phi^e$ is one of the three update functions that the define the GNN layer. Note that these are meant to be general expressions and you define $\phi^e$ for your specific GNN layer. 

Our molecular graphs are undirected, so how do we decide which node is receiving $\vec{v}_{rk}$ and which node is sending $\vec{v}_{sk}$? The individual $\vec{e}^{'}_k$ are aggregated in the next step as all the inputs into node $v_{rk}$. In our molecular graph, all bonds are both "inputs" and "outputs" from an atom (how else could it be?), so it makes sense to just view every bond as two directed edges: a C-H bond has an edge from C to H and an edge from H to C. In fact, our adjacency matrices already reflect that. There are two non-zero elements in them for each bond: one for C to H and one for H to C. Back to the original question, what is $\vec{v}_{rk}$ and $\vec{v}_{sk}$? We consider every element in the adjacency matrix (every $k$) and when we're on element $k = \{ij\}$, which is $A_{ij}$, then the receiving node is $j$ and the sending node is $i$. When we consider the companion edge $A_{ji}$, the receiving node is $i$ and the sending node is $j$. 

$\vec{e}^{'}_k$ is like the message from the GCN. Except it's more general: it can depend on the receiving node and the graph feature vector $\vec{u}$. The metaphor of a "message" doesn't quite apply, since a message cannot be affected by the receiver. Anyway, the new edge updates are then aggregated with the first aggregation function:

$$
\bar{e}^{'}_i = \rho^{e\rightarrow v}\left( E_i^{'}\right)
$$ (edge-aggregation)

where $\rho^{e\rightarrow v}$ is our defined function and $E_i^{'}$ represents stacking all $\vec{e}^{'}_k$ from edges  **into** node i. Having our aggregated edges, we can compute the node update:

$$
\vec{v}^{'}_i = \phi^v\left( \bar{e}^{'}_i, \vec{v}_i, \vec{u}\right)
$$ (node-update)

This concludes the usual steps of a GNN layer because we have new nodes and new edges. If you are updating the graph features ($\vec{u}$), the following additional steps may be defined:

$$
\bar{e}^{'} = \rho^{e\rightarrow u}\left( E^{'}\right)
$$ (edge-all-aggregation)

This equation aggregates all messages/aggregated edges across the whole graph. Then we can aggregate the new nodes across the whole graph:

$$
\bar{v}^{'} = \rho^{v\rightarrow u}\left( V^{'}\right)
$$ (node-all-aggregation)

Finally, we can compute the update to the graph feature vector as:

$$
\vec{u}^{'} = \phi^u\left( \bar{e}^{'},\bar{v}^{'}, \vec{u}\right)
$$ (global-update)


### Reformulating GCN into Battaglia equations

Let's see how the GCN is presented in this form. We first compute our neighbor messages for all possible neighbors using {eq}`edge-update`. Remember in the GCN, messages only depend on the senders.

```{margin}
Even though we use the "edge update" function, remember in a GCN we ignore 
the edge features. We only care edges for defining the connectivity of the graph.
```

$$
\vec{e}^{'}_k = \phi^e\left( \vec{e}_k, \vec{v}_{rk}, \vec{v}_{sk}, \vec{u}\right) = \vec{v}_{sk} \mathbf{W}
$$

To aggregate our messages coming into node $i$ in {eq}`edge-aggregation`, we average them.

$$
\bar{e}^{'}_i = \rho^{e\rightarrow v}\left( E_i^{'}\right) = \frac{1}{|E_i^{'}|}\sum E_i^{'}
$$

Our node update is then the activation {eq}`node-update`

$$
\vec{v}^{'}_i = \phi^v\left( \bar{e}^{'}_i, \vec{v}_i, \vec{u}\right) = \sigma(\bar{e}^{'}_i)
$$

we could include the self-loop above using $\sigma(\bar{e}^{'}_i + \vec{v}_i)$. The other functions are not used in a GCN, so those three completely define the GCN.

## The SchNet Architecture

One of the earliest and most popular GNNs is the SchNet network {cite}`schutt2018schnet`. It wasn't really recognized at publication time as a GNN, but its now recognized as one and you'll see it often used as a baseline model. A **baseline** model is a well-accepted and accurate model that is compared with.

```{margin} Baseline Models
A common piece of wisdom is if you want to solve a real problem with deep learning, you should read the most recent popular paper in an area and use the baseline they compare against instead of their proposed model. The reason is that a baseline model usually must be easy, fast, and well-tested, which is generally more important than being the most accurate
```

SchNet is for atoms represented as xyz coordinates (points) -- not as a molecular graph. All our previous examples used the underlying molecular graph as the input. In SchNet we will convert our xyz coodinates into a graph, so that we can apply a GNNN. SchNet was developed for predicting energies and forces from atom configurations without bond information. Thus, we need to first see how a set of atoms and their positions is converted into a graph. To get the nodes, we do a similar process as above and the atomic number is passed through an embedding layer, which is just means we assign a trainable vector to each atomic number (See {doc}`layers` for a review of embeddings). 

Getting the adjacency matrix is simple too: we just make every atom be connected to every atom. It might seem confusing what the point of using a GNN is, if we're just connecting everything. *It is because GNNs are permutation equivariant.* If we tried to do learning on the atoms as xyz coordinates, we would have weights depending on the ordering of atoms and probably fail to handle different numbers of atoms.

There is one more missing detail: where do the xyz coordinates go? We make the model depend on xyz coordinates by constructing the edge features from the xyz coordinates. The edge $\vec{e}$ between atoms $i$ and $j$ is computed purely from their distance $r$:

$$
e_k = \exp\left(-\gamma \left(r - \mu_k\right)^2\right)
$$ (rbf-edge)

where $\gamma$ is a hyperparameter (e.g., 10Å) and $\mu_k$ is an equally-space grid of scalars - like `[0, 5, 10, 15 , 20]`. The purpose of {eq}`rbf-edge` is similar to turning a category feature like atomic number or covalent bond type into a one-hot vector. We cannot do a one-hot vector though, because there is an infinite number of possible distances. Thus, we have a kind of "smoothing" that gives us a pseudo one-hot for distance. Let's see an example to get a sense of it:

In [ ]:
gamma = 1
mu = np.linspace(0, 10, 5)


def rbf(r):
    return np.exp(-gamma * (r - mu) ** 2)


print("input", 2)
print("output", np.round(rbf(2), 2))

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT


def ssp(x):
    return np.log(0.5 * np.exp(x) + 0.5)


def relu(x):
    return np.where(x < 0, np.zeros_like(x), x)


plt.figure()
x = np.linspace(-1.5, 1.5)
plt.axvline(x=0, linestyle="--", color="#AAA")
plt.axhline(y=0, linestyle="--", color="#AAA")
plt.plot(x, ssp(x), label="Shifted Softplus")
plt.plot(x, relu(x), label="ReLU")
plt.xlabel("$x$")
plt.ylabel("$\sigma(x)$")
plt.legend()
glue("softplus", plt.gcf(), display=False)

You can see that a distance of $r=2$ gives a vector with most of the activation for the $k = 1$ position - which corresponds to $\mu_1 = 2$. 

We have our nodes and edges and are close to defining the GNN update equations. We need a bit more notation. I'm going to use $h(\vec{x})$ to indicate a multilayer perceptron (MLP) -- basically a 1 to 2 dense layers neural network. The exact number of dense layers and when/where activation is used in these MLPs will be defined in the implementation, because it is not so important for understanding. Recall, the definition of a dense layer is

$$
h(\vec{x}) = \sigma\left(Wx + b\right)
$$

We'll also use a different activation function $\sigma$ called "shifted softplus" in SchNet: $\ln\left(0.5e^{x} + 0.5\right)$. You can see $\sigma(x)$ compared with the usual ReLU activation in {numref}`softplus`. The rationale for using shifted softplus is that it is smooth with-respect to its input, so it could be used to compute forces in a molecular dynamics simulation which requires taking smooth derivatives with respect to pairwise distances.

```{glue:figure} softplus
----
name: softplus
----
Comparison of the usual ReLU activation function and the shifted softplus used in the SchNet model.
```

Now, the GNN equations! The edge update equation {eq}`edge-update` is composed of two pieces. First, we run the incoming edge feature through an MLP and the atoms through an MLP. Then the result is run through an MLP:

$$
\vec{e}^{'}_k = \phi^e\left( \vec{e}_k, \vec{v}_{rk}, \vec{v}_{sk}, \vec{u}\right) =h_1\left(\vec{v}_{sk}\right) \cdot h_2\left(\vec{e}_k\right)
$$

The next equation is the edge aggregation equation, {eq}`edge-aggregation`. For SchNet, the edge aggregation is a sum over the neighbor atom features.

$$
\bar{e}^{'}_i = \sum E_i^{'}
$$

Finally, the node update equation for SchNet is:

$$
\vec{v}^{'}_i = \phi^v\left( \bar{e}^{'}_i, \vec{v}_i, \vec{u}\right) = \vec{v}_i + h_3\left(\bar{e}^{'}_i\right)
$$

The GNN updates are applied typically 3-6 times. Although we have an edge update equation, like in GCN we do not actually override the edges and keep them the same at each layer. The original SchNet was for predicting energies and forces, so a readout can be done using sum-pooling or any other strategy described above. 

These are sometimes changed, but in the original SchNet paper $h_1$ is one dense layer without activation, $h_2$ is two dense layers with activation, and $h_3$ is 2 dense layers with activation on the first and not the second.


```{admonition} What is SchNet?
The key GNN feature of a SchNet-like GNN are (1) use edge & node features in the edge update (message construction):

$$
\vec{e}^{'}_k = h_1(\vec{v}_{sk}) \cdot h_2(\vec{e}_k)
$$

where $h_i()$s are some trainable functions and (2) use a residue in the node update:

$$
\vec{v}^{'}_i = \vec{v}_i + h_3\left(\bar{e}^{'}_i\right)
$$

```

All the other details about how to featurize the edges, how deep $h_i$ is, what activation to choose, how to readout, and how to convert point clouds to graphs are about the specific SchNet model in {cite}`schutt2018schnet`. 

## SchNet Example: Predicting Space Groups

Our next example will be a SchNet model that predict space groups of points. Identifying the space group of atoms is an important part of crystal structure identification, and when doing simulations of crystallization. Our SchNet model will take as input points and output the predicted space group. This is a classification problem; specifically it is multi-class becase a set of points should only be in one space group. To simplify our plots and analysis, we will work in 2D where there are 17 possible space groups. 

Our data for this is a set of points from various point groups. The features are xyz coordinates and the label is the space group. We will not have multiple atom types for this problem. The hidden cell below loads the data and reshapes it for the example. 


In [ ]:
import gzip
import pickle
import urllib

urllib.request.urlretrieve(
    "https://github.com/whitead/dmol-book/raw/master/data/sym_trajs.pb.gz",
    "sym_trajs.pb.gz",
)
with gzip.open("sym_trajs.pb.gz", "rb") as f:
    trajs = pickle.load(f)

label_str = list(set([k.split("-")[0] for k in trajs]))

# now build dataset
def generator():
    for k, v in trajs.items():
        ls = k.split("-")[0]
        label = label_str.index(ls)
        traj = v
        for i in range(traj.shape[0]):
            yield traj[i], label


data = tf.data.Dataset.from_generator(
    generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32),
    ),
).shuffle(1000)

# The shuffling above is really important because this dataset is in order of labels!

val_data = data.take(100)
test_data = data.skip(100).take(100)
train_data = data.skip(200)

Let's take a look at a few examples from the dataset

```{admonition} The Data
:class: dropdown
This data was generated from {cite}`cox2022symmetric` and all points are constrained to match the space group exactly during a molecular dynamics simulation. The trajectories were NPT with a positive pressure and followed the procedure in that paper for Figure 2. The force field is Lennard-Jones with $\sigma=1$ and $\epsilon=1$
```

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(12, 8))
axs = axs.flatten()

# get a few example and plot them
for i, (x, y) in enumerate(data):
    if i == 20:
        break
    axs[i].plot(x[:, 0], x[:, 1], ".")
    axs[i].set_title(label_str[y.numpy()])
    axs[i].axis("off")

You can see that there is a variable number of points and a few examples for each space group. The goal is to infer those titles on the plot from the points alone.

### Building the graphs

We now need to build the graphs for the points. The nodes are all identical - so they can just be 1s (we'll reserve 0 in case we want to mask or pad at some point in the future). As described in the SchNet section above, the edges should be distance to every other atom. In most implementations of SchNet, we practically add a cut-off on either distance or maximum degree (edges per node). We'll do maximum degree for this work of 16.

I have a function below that is a bit sophisticated. It takes a matrix of point positions in arbitrary dimension and returns the distances and indices to the nearest `k` neighbors - exactly what we need. It uses some tricks from {doc}`../math/tensors-and-shapes`. However, it is not so important for you to understand this function. Just know it takes in points and gives us the edge features and edge nodes.

In [ ]:
# this decorator speeds up the function by "compiling" it (tracing it)
# to run efficienty
@tf.function(
    reduce_retracing=True,
)
def get_edges(positions, NN, sorted=True):
    M = tf.shape(input=positions)[0]
    # adjust NN
    NN = tf.minimum(NN, M)
    qexpand = tf.expand_dims(positions, 1)  # one column
    qTexpand = tf.expand_dims(positions, 0)  # one row
    # repeat it to make matrix of all positions
    qtile = tf.tile(qexpand, [1, M, 1])
    qTtile = tf.tile(qTexpand, [M, 1, 1])
    # subtract them to get distance matrix
    dist_mat = qTtile - qtile
    # mask distance matrix to remove zros (self-interactions)
    dist = tf.norm(tensor=dist_mat, axis=2)
    mask = dist >= 5e-4
    mask_cast = tf.cast(mask, dtype=dist.dtype)
    # make masked things be really far
    dist_mat_r = dist * mask_cast + (1 - mask_cast) * 1000
    topk = tf.math.top_k(-dist_mat_r, k=NN, sorted=sorted)
    return -topk.values, topk.indices

Let's see how this function works by showing the connections between points in one of our examples. I've hidden the code below. It shows some point's neighbors and connects them so you can get a sense of how a set of points is converted into a graph. The complete graph will have all points' neighborhoods.

In [ ]:
from matplotlib import collections

fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.flatten()
for i, (x, y) in enumerate(data):
    if i == 6:
        break
    e_f, e_i = get_edges(x, 8)

    # make things easier for plotting
    e_i = e_i.numpy()
    x = x.numpy()
    y = y.numpy()

    # make lines from origin to its neigbhors
    lines = []
    colors = []
    for j in range(0, x.shape[0], 23):
        # lines are [(xstart, ystart), (xend, yend)]
        lines.extend([[(x[j, 0], x[j, 1]), (x[k, 0], x[k, 1])] for k in e_i[j]])
        colors.extend([f"C{j}"] * len(e_i[j]))
    lc = collections.LineCollection(lines, linewidths=2, colors=colors)
    axs[i].add_collection(lc)
    axs[i].plot(x[:, 0], x[:, 1], ".")
    axs[i].axis("off")
    axs[i].set_title(label_str[y])
plt.show()

We will now add this function and the edge featurization of SchNet {eq}`rbf-edge` to get the graphs for the GNN steps.

In [ ]:
MAX_DEGREE = 16
EDGE_FEATURES = 8
MAX_R = 20

gamma = 1
mu = np.linspace(0, MAX_R, EDGE_FEATURES)


def rbf(r):
    return tf.exp(-gamma * (r[..., tf.newaxis] - mu) ** 2)


def make_graph(x, y):
    edge_r, edge_i = get_edges(x, MAX_DEGREE)
    edge_features = rbf(edge_r)
    return (tf.ones(tf.shape(x)[0], dtype=tf.int32), edge_features, edge_i), y[None]


graph_train_data = train_data.map(make_graph)
graph_val_data = val_data.map(make_graph)
graph_test_data = test_data.map(make_graph)

Let's examine one graph to see what it looks like. We'll slice out only the first nodes.

In [ ]:
for (n, e, nn), y in graph_train_data:
    print("first node:", n[1].numpy())
    print("first node, first edge features:", e[1, 1].numpy())
    print("first node, all neighbors", nn[1].numpy())
    print("label", y.numpy())
    break

### Implementing the MLPs

Now we can implement the SchNet model! Let's start with the $h_1,h_2,h_3$ MLPs that are used in the GNN update equations. In the SchNet paper these each had different numbers of layers and different decisions about which layers had activation. Let's create them now.

In [ ]:
def ssp(x):
    # shifted softplus activation
    return tf.math.log(0.5 * tf.math.exp(x) + 0.5)


def make_h1(units):
    return tf.keras.Sequential([tf.keras.layers.Dense(units)])


def make_h2(units):
    return tf.keras.Sequential(
        [
            tf.keras.layers.Dense(units, activation=ssp),
            tf.keras.layers.Dense(units, activation=ssp),
        ]
    )


def make_h3(units):
    return tf.keras.Sequential(
        [tf.keras.layers.Dense(units, activation=ssp), tf.keras.layers.Dense(units)]
    )

One detail that can be missed is that the weights in each MLP should change in each layer of SchNet. Thus, we've written the functions above to always return a new MLP. This means that a new set of trainable weights is generated on each call, meaning there is no way we could erroneously have the same weights in multiple layers.


### Implementing the GNN

Now we have all the pieces to make the GNN. This code will be very similar to the GCN example above, except we now have edge features. One more detail is that our readout will be an MLP as well, following the SchNet paper. The only change we'll make is that we want our output property to be (1) multi-class classification and (2) intensive (independent of number of atoms). So we'll end with an average (intensive) and end with an output vector of logits the size of our labels.

In [ ]:
class SchNetModel(tf.keras.Model):
    """Implementation of SchNet Model"""

    def __init__(self, gnn_blocks, channels, label_dim, **kwargs):
        super(SchNetModel, self).__init__(**kwargs)
        self.gnn_blocks = gnn_blocks

        # build our layers
        self.embedding = tf.keras.layers.Embedding(2, channels)
        self.h1s = [make_h1(channels) for _ in range(self.gnn_blocks)]
        self.h2s = [make_h2(channels) for _ in range(self.gnn_blocks)]
        self.h3s = [make_h3(channels) for _ in range(self.gnn_blocks)]
        self.readout_l1 = tf.keras.layers.Dense(channels // 2, activation=ssp)
        self.readout_l2 = tf.keras.layers.Dense(label_dim)

    def call(self, inputs):
        nodes, edge_features, edge_i = inputs
        # turn node types as index to features
        nodes = self.embedding(nodes)
        for i in range(self.gnn_blocks):
            # get the node features per edge
            v_sk = tf.gather(nodes, edge_i)
            e_k = self.h1s[i](v_sk) * self.h2s[i](edge_features)
            e_i = tf.reduce_sum(e_k, axis=1)
            nodes += self.h3s[i](e_i)
        # readout now
        nodes = self.readout_l1(nodes)
        nodes = self.readout_l2(nodes)
        return tf.reduce_mean(nodes, axis=0)

Remember that the key attributes of a SchNet GNN are the way that we use edge and node features. We can see the mixing of these two in the key line for computing the edge update (computing message values):

```python
e_k = self.h1s[i](v_sk) * self.h2s[i](edge_features)
```

followed by aggregation of the edges updates (pooling messages):

```python
e_i = tf.reduce_sum(e_k, axis=1)
```

and the node update

```python
nodes += self.h3s[i](e_i)
```

Also of note is how we go from node features to multi-classs. We use dense layers that get the shape per-node into the number of classes

```python
self.readout_l1 = tf.keras.layers.Dense(channels // 2, activation=ssp)
self.readout_l2 = tf.keras.layers.Dense(label_dim)
```

and then we take the average over all nodes

```python
return tf.reduce_mean(nodes, axis=0)
```

---

Let's give now use the model on some data.

In [ ]:
small_schnet = SchNetModel(3, 32, len(label_str))

In [ ]:
for x, y in graph_train_data:
    yhat = small_schnet(x)
    break
print(yhat.numpy())

The output is the correct shape and remember it is logits. To get a class prediction that sums to probability 1, we need to use a softmax:

In [ ]:
print("predicted class", tf.nn.softmax(yhat).numpy())

### Training

Great! It is untrained though. Now we can set-up training. Our loss will be cross-entropy from logits, but we need to be careful on the form. Our labels are integers - which is called "sparse" labels because they are not full one-hots. Mult-class classification is also known as categorical classification. Thus, the loss we want is sparse categorical cross entropy from logits.

In [ ]:
small_schnet.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics="sparse_categorical_accuracy",
)
result = small_schnet.fit(graph_train_data, validation_data=graph_val_data, epochs=20)

In [ ]:
plt.plot(result.history["sparse_categorical_accuracy"], label="training accuracy")
plt.plot(result.history["val_sparse_categorical_accuracy"], label="validation accuracy")
plt.axhline(y=1 / 17, linestyle="--", label="random")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

The accuracy is not great, but it looks like we could keep training. We have a very small SchNet here. Standard SchNet described in {cite}`schutt2018schnet` uses 6 layers and 64 channels and 300 edge features. We have 3 layers and 32 channels. Nevertheless, we're able to get some learning. Let's visually see what's going on with the trained model on some test data

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(12, 8))
axs = axs.flatten()

for i, ((x, y), (gx, _)) in enumerate(zip(test_data, graph_test_data)):
    if i == 20:
        break
    axs[i].plot(x[:, 0], x[:, 1], ".")
    yhat = small_schnet(gx)
    yhat_i = tf.math.argmax(tf.nn.softmax(yhat)).numpy()
    axs[i].set_title(f"True: {label_str[y.numpy()]}\nPredicted: {label_str[yhat_i]}")
    axs[i].axis("off")
plt.tight_layout()
plt.show()

We'll revisit this example later! One unique fact about this dataset is that it is *synthetic*, meaning there is no label noise. As discussed in {doc}`../ml/regression`, that removes the possibility of overfitting and leads us to favor high variance models. The goal of teaching a model to predict space groups is to apply it on real simulations or microscopy data, which will certainly have noise. We could have mimicked this by adding noise to the labels in the data and/or by randomly removing atoms to simulate defects. This would better help our model work in a real setting. 

## Current Research Directions

### Common Architecture Motifs and Comparisons

We've now seen message passing layer GNNs, GCNs, GGNs, and the generalized Battaglia equations. You'll find common motifs in the architectures, like gating, {doc}`attention`, and pooling strategies. For example, Gated GNNS (GGNs) can be combined with attention pooling to create Gated Attention GNNs (GAANs){cite}`zhang2018gaan`. GraphSAGE is a similar to a GCN but it samples when pooling, making the neighbor-updates of fixed dimension{cite}`hamilton2017inductive`. So you'll see the suffix "sage" when you sample over neighbors while pooling. These can all be represented in the Battaglia equations, but you should be aware of these names. 

The enormous variety of architectures has led to work on identifying the "best" or most general GNN architecture {cite}`dwivedi2020benchmarking,errica2019fair,shchur2018pitfalls`. Unfortunately, the question of which GNN architecture is best is as difficult as "what benchmark problems are best?" Thus there are no agreed-upon conclusions on the best architecture. However, those papers are great resources on training, hyperparameters, and reasonable starting guesses and I highly recommend reading them before designing your own GNN. There has been some theoretical work to show that simple architectures, like GCNs, cannot distinguish between certain simple graphs {cite}`xu2018powerful`. How much this practically matters depends on your data. Ultimately, there is so much variety in hyperparameters, data equivariances, and training decisions that you should think carefully about how much the GNN architecture matters before exploring it with too much depth. 

### Nodes, Edges, and Features

You'll find that most GNNs use the node-update equation in the Battaglia equations but do not update edges. For example, the GCN will update nodes at each layer but the edges are constant. Some recent work has shown that updating edges can be important for learning when the edges have geometric information, like if the input graph is a molecule and the edges are distance between the atoms {cite}`klicpera2019directional`. As we'll see in the chapter on equivariances ({doc}`../dl/data`), one of the key properties of neural networks with point clouds (i.e., Cartesian xyz coordinates) is to have rotation equivariance. {cite}`klicpera2019directional` showed that you can achieve this if you do edge updates and encode the edge vectors using a rotation equivariant basis set with spherical harmonics and Bessel functions. These kind of edge updating GNNs can be used to predict protein structure {cite}`jing2020learning`.

Another common variation on node features is to pack more into node features than just element identity. In many examples, you will see people inserting valence, elemental mass, electronegativity, a bit indicating if the atom is in a ring, a bit indicating if the atom is aromatic, etc. Typically these are unnecessary, since a model should be able to learn any of these features which are computed from the graph and node elements. However, we and others have empirically found that some can help, specifically indicating if an atom is in a ring {cite}`li2020graph`. Choosing extra features to include though should be at the bottom of your list of things to explore when designing and using GNNs.

### Beyond Message Passing

One of the common themes of GNN research is moving "beyond message passing," where message passing is the message construction, aggregation, and node update with messages. Some view this as impossible -- claiming that all GNNs can be recast as message passing {cite}`velivckovic2022message`. Another direction is on disconnecting the underlying graph being input to the GNN and the graph used to compute updates. We sort of saw this above with SchNet, where we restricted the maximum degree for the message passing. More useful are ideas like "lifting" the graphs into more structured objects like simplicial complexes {cite}`bodnar2021weisfeiler`. Finally, you can also choose where to send the messages beyond just neighbors {cite}`thiede2021autobahn`. For example, all nodes on a path could communicate messages or all nodes in a clique. 

### Do we need graphs?

It is possible to convert a graph into a string if you're working with an adjacency matrix without continuous values. Molecules specifically can be converted into a string. This means you can use layers for sequences/strings (e.g., recurrent neural networks or 1D convolutions) and avoid the complexities of a graph neural network. SMILES is one way to convert molecular graphs into strings. With SMILES, you cannot predict a per-atom quantity and thus a graph neural network is required for atom/bond labels. However, the choice is less clear for per-molecule properties like toxicity or solubility. There is no consensus about if a graph or string/SMILES representation is better. SMILES can exceed certain graph neural networks in accuracy on some tasks. SMILES is typically better on generative tasks. Graphs obviously beat SMILES in label representations, because they have granularity of bonds/edges.  We'll see how to model SMILES in {doc}`NLP`, but it is an open question of which is better.


### Stereochemistry/Chiral Molecules

Stereochemistry is fundamentally a 3D property of molecules and thus not present in the covalent bonding. It is measured experimentally by seeing if molecules rotate polarized light and a molecule is called chiral or "optically active" if it is experimentally known to have this property. Stereochemistry is the categorization of how molecules can preferentially rotate polarized light through asymmetries with respect to their mirror images. In organic chemistry, the majority of stereochemistry is of enantiomers. Enantiomers are "handedness" around specific atoms called chiral centers which have 4 or more different bonded atoms. These may be treated in a graph by indicating which nodes are chiral centers (nodes) and what their state or mixture of states (racemic) are. This can be treated as an extra processing step. Amino acids and thus all proteins are entaniomers with only one form present. This chirality of proteins means many drug molecules can be more or less potent depending on their stereochemistry. 


```{figure} ../_static/images/helicene.mp4
----
name: helicene
width: 500px
class: autoplay-video
----
This is a molecule with axial stereochemistry. Its small helix could be either left or right-handed. 
```

Adding node labels is not enough generally. Molecules can interconvert between stereoisomers at chiral centers through a process called tautomerization. There are also types of stereochemistry that are not at a specific atom, like rotamers that are around a bond. Then there is stereochemistry that involves multiple atoms like axial helecene. As shown in {numref}`helicene`, the molecule has no chiral centers but is "optically active" (experimentally measured to be chiral) because of its helix which can be left- or right-handed. 

## Relevant Videos

### Intro to GNNs

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/uF53xsT7mjc" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Overview of GNN with Molecule, Compiler Examples

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/zCEYiCxrL_0" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>


## Chapter Summary 

* Molecules can be represented by graphs by using one-hot encoded feature vectors that show the elemental identity of each node (atom) and an adjacency matrix that show immediate neighbors (bonded atoms).
* Graph neural networks are a category of deep neural networks that have graphs as inputs.
* One of the early GNNs is the Kipf & Welling GCN. The input to the GCN is the node feature vector and the adjacency matrix, and returns the updated node feature vector. The GCN is permutation invariant because it averages over the neighbors. 
* A GCN can be viewed as a message-passing layer, in which we have senders and receivers. Messages are computed from neighboring nodes, which when aggregated update that node. 
* A gated graph neural network is a variant of the message passing layer, for which the nodes are updated according to a gated recurrent unit function. 
* The aggregation of messages is sometimes called pooling, for which there are multiple reduction operations. 
* GNNs output a graph. To get a per-atom or per-molecule property, use a readout function. The readout depends on if your property is intensive vs extensive 
* The Battaglia equations encompasses almost all GNNs into a set of 6 update and aggregation equations.
* You can convert xyz coordinates into a graph and use a GNN like SchNet 

## Cited References

```{bibliography}
:style: unsrtalpha
:filter: docname in docnames
```